In [56]:
import backtrader as bt
import talib as ta
import pandas as pd
import json
from futu import *
import time
import numpy as np
import math
import datetime
import pandas as pd
import backtrader as bt
import backtrader.indicators as btind
import matplotlib
matplotlib.use('nbagg')  # 适用于 Jupyter 的交互式后端
import matplotlib.pyplot as plt  # 导入 matplotlib 确保显示
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 300

In [ ]:
from futu import *

def get_stock_pool(pool_name="全部"):
    quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
    ret, data = quote_ctx.get_user_security(pool_name)
    if ret == RET_OK:
        # print(data)
        if data.shape[0] > 0:  # 如果自选股列表不为空
            res = data['code'].values.tolist()
        else:
            res = []
    else:
        print('error:', data)
        res = []
    quote_ctx.close()
    return res

In [6]:
stock_pool = get_stock_pool()

2025-10-22 15:20:20,724 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=3, host=127.0.0.1, port=11111, user_id=31759416
2025-10-22 15:20:20,790 | 42768 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=3


In [ ]:
df = pd.read_parquet('future/data/kline_data.parquet')
df.head()
df.dtypes

In [46]:
def get_all_plate(market: Market):
    quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
    ret, data = quote_ctx.get_plate_list(market, Plate.CONCEPT)
    res = {}
    if ret == RET_OK:
        for index, row in data.iterrows():
            res[row['plate_name']] = row['code']
    else:
        print('error:', data)
    quote_ctx.close()
    return res


In [47]:
all_plates = get_all_plate(Market.SH)

2025-10-22 19:56:48,107 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=11, host=127.0.0.1, port=11111, user_id=31759416


In [19]:
need_plates_topic = ["稀土概念", "人工智能", "新能源汽车", "固态电池", "白酒", "银行保险证券", "芯片", "国产替代", "机器人"]

In [ ]:
res = []
for name, code in all_plates.items():
    res.append(name)
remp = ','.join(res)
print(len(res))

752


In [25]:
from openai import OpenAI

# for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
def get_ai_recommendation(prompt: str) -> str:
    client = OpenAI(api_key="sk-8c887e6454f741cc9553da2be62487af", base_url="https://api.deepseek.com")
    response = client.chat.completions.create(
        model="deepseek-chat",
            messages=[
                {"role": "system", "content": "你是一个专业的股票助手。"},
                {"role": "user", "content": prompt},
            ],
            max_tokens=1024,
        temperature=0.7,
        stream=False
    )
    return response.choices[0].message.content


In [38]:
prompt = f"""请从以下找出的概念板块中找出和列出的主题相关的概念板块，并以json格式输出
概念板块:
{remp}
主题:
{need_plates_topic}
输出示例：
```json{{
    "concept": [
        "稀土概念",
        "AI",
    ]
}}```
"""

In [39]:
plates = get_ai_recommendation(prompt)

In [42]:
import json_repair

In [43]:
decoded_object = json_repair.loads(plates)

In [49]:
target_plate_codes = []
for item in decoded_object['concept']:
    target_plate_codes.append(all_plates[item])

In [57]:
from typing import Dict
def filter_stocks_by_kdj_criteria(market: Market, plate_code: str) -> Dict[str, str]:
    """
    根据KDJ指标筛选股票
    
    参数:
        market: 市场类型，如 Market.SH, Market.SZ
        plate_code: 板块代码
    
    返回:
        Dict[str, str]: 股票代码到股票名称的映射字典
    """
    # 存储筛选结果
    stock_results = {}
    
    try:
        # 初始化行情上下文
        quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
        
        # 创建价格过滤器（价格范围2-1000）
        simple_filter = SimpleFilter()
        simple_filter.filter_min = 2
        simple_filter.filter_max = 1000
        simple_filter.stock_field = StockField.CUR_PRICE
        simple_filter.is_no_filter = True
        
        # 创建模式过滤器（KDJ低位金叉）
        pattern_filter = PatternFilter()
        pattern_filter.ktype = KLType.K_DAY
        pattern_filter.stock_field = PatternField.KDJ_GOLD_CROSS_LOW
        pattern_filter.is_no_filter = True
        
        # 创建自定义指标过滤器（KDJ的J值小于-5）
        custom_filter = CustomIndicatorFilter()
        custom_filter.ktype = KLType.K_DAY
        custom_filter.stock_field1 = StockField.KDJ_J
        custom_filter.stock_field1_para = [9, 3, 3]
        custom_filter.relative_position = RelativePosition.LESS
        custom_filter.value = -5
        custom_filter.is_no_filter = False
        
        # 分页查询
        nBegin = 0
        last_page = False
        ret_list = list()
        
        while not last_page:
            nBegin += len(ret_list)
            # 使用传入的market和plate_code参数
            ret, ls = quote_ctx.get_stock_filter(
                market=market, 
                filter_list=[simple_filter, pattern_filter, custom_filter], 
                begin=nBegin, 
                plate_code=plate_code
            )  
            
            if ret == RET_OK:
                last_page, all_count, ret_list = ls
                print(f'all count = {all_count}')
                
                # 收集股票代码和名称
                for item in ret_list:
                    stock_results[item.stock_code] = item.stock_name
            else:
                print(f'error: {ls}')
                break
            
            # 添加时间间隔，避免触发限频
            time.sleep(3)
            
    except Exception as e:
        print(f'筛选过程中发生错误: {e}')
    finally:
        # 确保关闭连接
        try:
            if 'quote_ctx' in locals():
                quote_ctx.close()
        except:
            pass
    
    return stock_results

In [58]:
for plate_code in target_plate_codes:
    res = filter_stocks_by_kdj_criteria(Market.SH, plate_code)
    print(res)
    break

2025-10-22 20:04:14,201 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=14, host=127.0.0.1, port=11111, user_id=31759416
筛选过程中发生错误: name 'PatternField' is not defined
{}
